In [ ]:
import spectHR as cs
import pyhrv
from ipywidgets import Tab, Output
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')
cs.handler.show_logs()

In [ ]:
DataSet = cs.SpectHRDataset("SUB_018.xdf", use_webdav=True, reset = False)
DataSet = cs.borderData(DataSet)
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})
if not hasattr(DataSet, 'RTops'):
    DataSet = cs.calcPeaks(DataSet)

In [ ]:
%matplotlib widget
GUI = cs.prepPlot(DataSet)

preProcessing =  Output()
with preProcessing:
    display(GUI)

poincarePlot = Output()
psdPlot = Output()
descriptives = Output()
Gantt = Output()

import pandas as pd

App = Tab(children=[preProcessing, poincarePlot, descriptives, psdPlot, Gantt])

App.set_title(0, "PreProcessing")
App.set_title(1, "Poincare")
App.set_title(2, "Descriptives")
App.set_title(3, "PSD")
App.set_title(4, "Epochs")

# This function is called everytime a tab is selected in the App. The value of change['new'] holds the index to the selected tab.
def on_tab_change(change):
    if change['name'] == 'selected_index':        
        if change['new'] == 1:  # Poincare selected
            with poincarePlot:
                poincarePlot.clear_output()
                display(cs.poincare(DataSet))
            
        if change['new'] == 2: # selected the descriptives tab
            with descriptives:
                descriptives.clear_output()
                display(cs.explode(DataSet)
                        .groupby('epoch')['ibi']
                        .agg([
                            ('N', len),
                            ('mean', 'mean'),
                            ('std', 'std'),
                            ('min', 'min'),
                            ('max', 'max'),
                            ('rmssd', lambda x: pyhrv.time_domain.rmssd(x)[0]),  # Extracting the first element
                            ('sdnn', lambda x: pyhrv.time_domain.sdnn(x)[0]),
                            ('sdsd', cs.Tools.Params.sdsd),
                            ('sd1', cs.Tools.Params.sd1),
                            ('sd2', cs.Tools.Params.sd2),
                            ('sd_ratio', cs.Tools.Params.sd_ratio),
                            ('ellipse_area', cs.ellipse_area)
                        ]))
                
        if change['new'] == 3:  # psd selected
            with psdPlot:
                psdPlot.clear_output()    
                Data = cs.explode(DataSet)
                psd_Parameters = Data.groupby('epoch')[Data.columns.tolist()].apply(cs.welch_psd)
                display(psd_Parameters)

        if change['new'] == 4:  # gantt selected 
            with Gantt:
                Gantt.clear_output()
                display(cs.gantt(DataSet, labels=False))
                
    DataSet.save()
                
# Attach observer to detect tab switch
App.observe(on_tab_change, names='selected_index')

display(App)

In [ ]:
cs.welch_psd(DataSet.RTops, interpolate = True)
cs.welch_psd(DataSet.RTops, interpolate = False)

In [ ]:
psd_Parameters